In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/UniAcco - Twitter/tweets.csv")

In [ ]:
df

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,user_name,Tweets
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
999994,4,1879942807,Thu May 21 23:36:19 PDT 2009,NO_QUERY,divabat,"@healingsinger thank you, i needed that"
999995,4,1879942922,Thu May 21 23:36:20 PDT 2009,NO_QUERY,nick1975,@vactress http://bit.ly/cADea Maybe this is m...
999996,4,1879942975,Thu May 21 23:36:21 PDT 2009,NO_QUERY,znmeb,"@Brat13 Hell, Windows 7 will be out of my pric..."
999997,4,1879943113,Thu May 21 23:36:22 PDT 2009,NO_QUERY,virmani,@jigardoshi neah.. i wish! just reminiscing r...


In [ ]:
df.isna()

In [ ]:
#Check for null values
df.isnull().sum()

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
#Remove columns which is not useful
df.drop(['0','1467810369', 'Mon Apr 06 22:19:45 PDT 2009', 'NO_QUERY', 'user_name'], axis=1, inplace=True)

In [ ]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
#Convert everthing to lowercase
df['Tweets'] = df['Tweets'].str.lower()

In [ ]:
#Remove punctuation 
df['Tweets'] = df['Tweets'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
#Remove stopwords
stop_words = set(stopwords.words('english'))
df['Tweets'] = df['Tweets'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word not in stop_words]))
df

In [ ]:
df.to_csv('processed_data.csv', index=False)

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

# Load the pre-trained model and tokenizer
model_name = "cardiffnlp/tweet-topic-21-multi"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

def predict_topics(df):
    # Split the text column on newline characters
    texts = df['Tweets'].astype(str).str.split('\n')
    # Flatten the list of texts
    texts = [text for sublist in texts for text in sublist]
    # Remove any empty strings
    texts = [text for text in texts if text]

    # Tokenize the texts
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='tf')
    
    # Predict the topics of the tweets
    outputs = model(inputs)
    predicted_topics = tf.argmax(outputs.logits, axis=1)
    
    # Map the predicted topic IDs to labels
    predicted_topic_labels = predicted_topics.numpy().tolist()
    predicted_topic_labels = [topic_mapping.get(label) for label in predicted_topic_labels]
    
    # Add the predicted topic labels to the DataFrame
    df['predicted_topic'] = predicted_topics

    return df


# Define the mapping dictionary for topic IDs to labels
topic_mapping = {
    0: 'arts_&_culture',
    1: 'business_&_entrepreneurs',
    2: 'celebrity_&_pop_culture',
    3: 'diaries_&_daily_life',
    4: 'family',
    5: 'fashion_&_style',
    6: 'film_tv_&_video',
    7: 'fitness_&_health',
    8: 'food_&_dining',
    9: 'gaming',
    10: 'learning_&_educational',
    11: 'music',
    12: 'news_&_social_concern',
    13: 'other_hobbies',
    14: 'relationships',
    15: 'science_&_technology',
    16: 'sports',
    17: 'travel_&_adventure',
    18: 'youth_&_student_life'
}

# Load the CSV file into a DataFrame
df = pd.read_csv('/content/processed_data.csv', nrows=15000)

# Predict the topics of the tweets
df = predict_topics(df)

# Print the predicted topics of the tweets
print(df[['Tweets', 'predicted_topic']])


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/tweet-topic-21-multi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


                                                 Tweets  predicted_topic
0                wan na see fleet foxes coachella today               11
1     jonasbrothers httptwitpiccom3br5p sweet pic gu...                2
2                            wishing louisville thunder               11
3     town propper bored auntys barbie haha difficul...                3
4     went bike ride nothing extraordinarily excitin...                3
...                                                 ...              ...
1995               aubreyoday hey aubrey ya ya dk split                2
1996  twaiting wife come stairs knock tivo leaves to...                3
1997                          yer hgistory presetnation               11
1998                          really miss talking missy                3
1999  dinner ok mussels appetizer followed chicken s...                8

[2000 rows x 2 columns]


In [ ]:
df.to_csv('predicted.csv', index=False)

LABEL THE PREDICTED VALUE

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/UniAcco - Twitter/predicted.xlsx')

In [ ]:
import pandas as pd

# Define the mapping dictionary for topic IDs to labels
topic_mapping = {
    0: 'arts_&_culture',
    1: 'business_&_entrepreneurs',
    2: 'celebrity_&_pop_culture',
    3: 'diaries_&_daily_life',
    4: 'family',
    5: 'fashion_&_style',
    6: 'film_tv_&_video',
    7: 'fitness_&_health',
    8: 'food_&_dining',
    9: 'gaming',
    10: 'learning_&_educational',
    11: 'music',
    12: 'news_&_social_concern',
    13: 'other_hobbies',
    14: 'relationships',
    15: 'science_&_technology',
    16: 'sports',
    17: 'travel_&_adventure',
    18: 'youth_&_student_life'
}

# Map the topic IDs to labels
df['predicted_topic'] = df['predicted_topic'].map(topic_mapping)

# Print the DataFrame with topic labels
print(df)


In [ ]:
df.to_csv('output_data.csv', index=False)